In [ ]:
%matplotlib inline
from checkmate.core.graph_builder import gen_linear_graph
from checkmate.core.solvers.poet_solver import extract_costs_from_dfgraph, solve_poet_cvxpy
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm
import logging
import pandas as pd
sns.set('notebook')
sns.set_style('dark')

In [ ]:
!mkdir -p data/poet_sweep
data = []
for cost_factor in tqdm([0.1, 1.0, 2.0, 4.0, 5.0, 8.0, 16.0]):
    for budget in [1.0, 2.0, 3.0, 4.0]:
        try:
            g = gen_linear_graph(16)
            cpu_cost, page_in_cost, page_out_cost = extract_costs_from_dfgraph(g, cost_factor)
            solution = solve_poet_cvxpy(g, budget, cpu_cost, page_in_cost, page_out_cost, solver_override="GUROBI", verbose=True)

            data.append(dict(cost_factor=cost_factor, budget=budget, solution=solution, dfgraph=g,
                             cpu_cost=cpu_cost, page_in_cost=page_in_cost, page_out_cost=page_out_cost))

            plt.figure()
            fig, axarr = plt.subplots(1, 5, figsize=(25, 5))
            for arr, ax, name in zip(solution, axarr, ['R', 'S_RAM', 'S_SD',  'M_sd2ram', 'M_ram2sd']):
                ax.invert_yaxis()
                ax.pcolormesh(arr, cmap="Greys", vmin=0, vmax=1)
                ax.set_title(name)
            fig.savefig('data/poet_sweep/{}_{}.png'.format(cost_factor, budget))
        except Exception as e:
            logging.exception(e)
            continue

In [ ]:
def featurize_row(data_row):
    out_vec = dict(cost_factor=data_row['cost_factor'], budget=data_row['budget'])
    R, S_RAM, S_SD, M_sd2ram, M_ram2sd = data_row['solution']
    out_vec['total_compute_runtime'] = np.sum(R @ data_row['cpu_cost'])
    out_vec['total_page_cost'] = (np.sum(M_sd2ram @ data_row['page_out_cost']) + np.sum(M_ram2sd @ data_row['page_in_cost'])) / data_row['cost_factor']
    return out_vec
df = pd.DataFrame(map(featurize_row, data))

In [ ]:
for budget in [2.0, 3.0, 4.0]:
    df[df['budget'] == budget].plot(x='cost_factor', y=['total_compute_runtime', 'total_page_cost'], kind='bar', stacked=True)

In [ ]:
df[df['cost_factor'] == 2.0].plot(x='budget', y=['total_compute_runtime', 'total_page_cost'], kind='bar', stacked=True)

In [ ]:
data[0]